In [112]:
import itertools
import pickle
import numpy as np
from scipy.stats import poisson
import time
import sys
import pandas as pd

N_PHASE = 0 # int(sys.argv[1])
#import multiprocessing as mp

eps = np.log2(.0001)
max_count = 25+1
kmer_length = 100


BAM_MAPPINGS_FILE = '/home/groups/dpwall/briannac/general_data/bam_mappings.csv'

# Construct average k-mer depth dictionary.
bam_mappings = pd.read_csv(BAM_MAPPINGS_FILE, sep='\t', index_col=1)
bam_mappings = bam_mappings[bam_mappings['status']=='Passed_QC_analysis_ready']
PARENT_DIR = '/home/groups/dpwall/briannac/blood_microbiome/'
ihart_flagstat_file = PARENT_DIR + 'data/ihart_flagstat.csv'
flagstat = pd.read_csv(ihart_flagstat_file, index_col=0)
flagstat = flagstat.loc[set(flagstat.index).intersection(bam_mappings.index)]
bam_mappings = bam_mappings.loc[flagstat.index]
total_mapped_reads = flagstat.ProperPair*((flagstat.Total_Reads-flagstat.Supplementary-flagstat.Duplicates)/flagstat.Total_Reads)
avg_coverage = total_mapped_reads*150/(6.27e9)#*(bam_mappings.sex_numeric.astype(float)==1.0) + 6.37e9*(bam_mappings.sex_numeric.astype(float)==2.0))
avg_n_100mers = (150-kmer_length)/(150/avg_coverage)
kmer_depth_dict = {k:avg_n_100mers[k] for k in avg_n_100mers.keys()}


with open('/home/groups/dpwall/briannac/alt_haplotypes/data/phasings/phased_fams/phased_fams_all.pickle', 'rb') as f:
    phased_fam_dict = pickle.load(f)

    
    
all_family_phasings = set()
for f in phased_fam_dict:
    fam_phases = phased_fam_dict[f]
    all_family_phasings = all_family_phasings.union(set([tuple(v) for v in fam_phases.values]))
    
    
poisson_cache = [[], []]
avg_kmer_depth=np.mean(list(kmer_depth_dict.values()))

poisson_cache[0] = [poisson.pmf(k=k, mu=avg_kmer_depth) for k in range(max_count)]
poisson_cache[1] = [poisson.pmf(k=k, mu=2*avg_kmer_depth) for k in range(max_count)]

def cached_poisson_pmf(k,g):
    return min(max(poisson_cache[g-1][k], eps), 1-3*eps)

if True:
    phases_ch = list(all_family_phasings)[N_PHASE]
    possible_gs = [(0,0),(0,1), (1,0), (1,1)]
    family_probability_cache = dict()
    for k_m in range(max_count): # Iterate through possible k-mer counts for mom.  (up to 30)
        print(k_m/max_count)
        for k_p in range(max_count): # Iterate through possible k-mer counts for dad (up to 30.)
            for k_cs in itertools.combinations_with_replacement(range(max_count), len(phases_ch)): # Iterate through possible k-mer counts for children (up to 30.)
                family_probability_cache[(k_m, k_p, k_cs, phases_ch)] = np.log2(sum(
                    [cached_poisson_pmf(k_m,sum(g_m))*
                     cached_poisson_pmf(k_p,sum(g_p))*
                     np.prod([cached_poisson_pmf(k_c, g_m[phase_ch[0]]+g_p[phase_ch[1]]) for k_c, phase_ch in zip(k_cs, phases_ch)]
                            ) for g_p in possible_gs for g_m in possible_gs]))
    #return family_probability_cache

#pool = mp.Pool(mp.cpu_count())

#dicts = pool.map(Probabilities, list(all_family_phasings))
#pool.close()
#family_probability_dict = Probabilities()
#for d in dicts:
#    family_probability_dict.update(d)

#with open('/home/groups/dpwall/briannac/alt_haplotypes/intermediate_files/localize/family_probability_cache_%i.pickle' % N_PHASE, 'wb') as f:
#        pickle.dump(family_probability_dict, f, protocol=pickle.HIGHEST_PROTOCOL)



0.0


KeyboardInterrupt: 

In [108]:
import pickle

family_probability_dict = {}
for N_PHASE in range(250):
    print(N_PHASE)
    with open('/home/groups/dpwall/briannac/alt_haplotypes/intermediate_files/localize/family_probability_cache_%i.pickle' % N_PHASE, 'rb') as f:
        family_probability_dict = {**family_probability_dict, **(pickle.load(f))}

with open('/home/groups/dpwall/briannac/alt_haplotypes/data/localize/family_probability_cache.pickle', 'wb') as f:
    pickle.dump(family_probability_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

68
69
74
83


In [114]:
family_probability_dict

{(0, 0, (0, 0), ((1, 1), (0, 0))): -32.87501319925711,
 (0, 0, (0, 1), ((1, 1), (0, 0))): -30.321592552048088,
 (0, 0, (0, 2), ((1, 1), (0, 0))): -28.76813742764276,
 (0, 0, (0, 3), ((1, 1), (0, 0))): -27.799575856981,
 (0, 0, (0, 4), ((1, 1), (0, 0))): -27.245913921295273,
 (0, 0, (0, 5), ((1, 1), (0, 0))): -27.01390447043902,
 (0, 0, (0, 6), ((1, 1), (0, 0))): -27.04437867899872,
 (0, 0, (0, 7), ((1, 1), (0, 0))): -27.296145706915965,
 (0, 0, (0, 8), ((1, 1), (0, 0))): -27.738366140909363,
 (0, 0, (0, 9), ((1, 1), (0, 0))): -28.346158112581957,
 (0, 0, (0, 10), ((1, 1), (0, 0))): -29.097363818768944,
 (0, 0, (0, 11), ((1, 1), (0, 0))): -29.96934954767178,
 (0, 0, (0, 12), ((1, 1), (0, 0))): -30.935127767476235,
 (0, 0, (0, 13), ((1, 1), (0, 0))): -31.958956473195688,
 (0, 0, (0, 14), ((1, 1), (0, 0))): -32.994164042874054,
 (0, 0, (0, 15), ((1, 1), (0, 0))): -33.98995430150221,
 (0, 0, (0, 16), ((1, 1), (0, 0))): -34.911301202222795,
 (0, 0, (0, 17), ((1, 1), (0, 0))): -35.7580135159

In [59]:
def cached_poisson_pmf(g,k,mu):
    return min(max(poisson_cache[g-1][(k,mu)], eps), 1-3*eps)

fam = list(family_info.index)[N_PHASE]
sampleid_mom = family_info.loc[fam].mother_sample
sampleid_dad = family_info.loc[fam].father_sample
sampleid_children = family_info.loc[fam].sib_samples

possible_gs = [(0,0),(0,1), (1,0), (1,1)]
family_probability_cache = dict()
for k_m in range(int(3*kmer_depth_dict[sampleid_mom])): # Iterate through possible k-mer counts for mom.  (up to 30)
    for k_p in range(int(3*kmer_depth_dict[sampleid_dad])): # Iterate through possible k-mer counts for dad (up to 30.)
        for phases_ch in set([tuple(v) for v in phased_fam_dict[fam].values]):
            for k_cs in itertools.product(*[[i for i in range(int(3*kmer_depth_dict[s]))] for s in sampleid_children]):#, len(phases_ch)): # Iterate through possible k-mer counts for children (up to 30.)
                family_probability_cache[(fam, k_m, k_p, k_cs, phases_ch)] = np.log2(sum(
                    [cached_poisson_pmf(sum(g_m),k_m, kmer_depth_dict[sampleid_mom])*
                        cached_poisson_pmf(sum(g_p), k_p, kmer_depth_dict[sampleid_dad])*
                        np.prod([cached_poisson_pmf(g_m[phase_ch[0]]+g_p[phase_ch[1]], k_c, kmer_depth_dict[id_c]) for id_c, k_c, phase_ch in zip(sampleid_children, k_cs, phases_ch)]
                                ) for g_p in possible_gs for g_m in possible_gs]))
        print(k_p/max_count, len(family_probability_cache))
    print(k_m/max_count)

family_probability_dict = family_probability_cache

#with open('/home/groups/dpwall/briannac/alt_haplotypes/intermediate_files/localize/family_probability_cache_%i.pickle' % N_PHASE, 'wb') as f:
#        pickle.dump(family_probability_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

0.0 78336
0.038461538461538464 156672


KeyboardInterrupt: 

In [72]:
len(itertools.product(*[[i for i in range(int(3*kmer_depth_dict[s]))] for s in sampleid_children]))

NameError: name 'sampleid_children' is not defined

In [19]:
cached_poisson_pmf(g_m[phase_ch[0]]+g_p[phase_ch[1]], k_c, kmer_depth_dict[id_c])

NameError: name 'g_m' is not defined

In [17]:
family_probability_dict

NameError: name 'family_probability_dict' is not defined

In [17]:
all_family_phasings = set()
for f in phased_fam_dict:
    fam_phases = phased_fam_dict[f]
    all_family_phasings = all_family_phasings.union(set([tuple(v) for v in fam_phases.values]))


In [99]:
phased_fam_dict

{'AU2298':                        09C79839 09C79840 09C79841
 01.000000001.000597032   (0, 0)   (0, 0)   (0, 1)
 01.000597032.018853519   (0, 0)   (0, 0)   (0, 0)
 01.018853519.028294219   (0, 0)   (0, 0)   (0, 1)
 01.028295025.061292340   (0, 0)   (0, 0)   (0, 1)
 01.061292340.062376231   (0, 0)   (0, 0)   (1, 1)
 ...                         ...      ...      ...
 XX.000000001.001762856   (0, 0)   (0, 0)   (0, 0)
 XX.001762856.047618069   (0, 0)   (0, 0)   (1, 0)
 XX.047618069.098970324   (0, 0)   (0, 0)   (0, 0)
 XX.098970324.136119349   (0, 0)   (0, 0)   (1, 0)
 XX.136119349.156040895   (0, 0)   (0, 0)   (0, 0)
 
 [277 rows x 3 columns],
 'AU0063':                        03C16982 03C16983 03C16984
 01.000000001.026703060   (0, 0)   (0, 0)   (0, 1)
 01.026703060.056555273   (0, 0)   (0, 0)   (1, 1)
 01.056555273.061208641   (0, 0)   (0, 0)   (1, 0)
 01.061208641.074008899   (0, 0)   (0, 1)   (1, 1)
 01.074008899.101805084   (0, 0)   (1, 1)   (0, 1)
 ...                         ...   

In [8]:
phased_fam_dict[list(phased_fam_dict.keys())[0]]

,09C79839,09C79840,09C79841
01.000000001.000597032,"(0, 0)","(0, 0)","(0, 1)"
01.000597032.018853519,"(0, 0)","(0, 0)","(0, 0)"
01.018853519.028294219,"(0, 0)","(0, 0)","(0, 1)"
01.028295025.061292340,"(0, 0)","(0, 0)","(0, 1)"
01.061292340.062376231,"(0, 0)","(0, 0)","(1, 1)"
...,...,...,...
XX.000000001.001762856,"(0, 0)","(0, 0)","(0, 0)"
XX.001762856.047618069,"(0, 0)","(0, 0)","(1, 0)"
XX.047618069.098970324,"(0, 0)","(0, 0)","(0, 0)"
XX.098970324.136119349,"(0, 0)","(0, 0)","(1, 0)"


In [1]:
poisson_cache = [[], []]
#avg_kmer_depth=avg_kmer_depth
poisson_cache[0] = {(k,mu): poisson.pmf(k=k, mu=mu) for k in range(max_count) for mu in kmer_depth_dict.values()}
poisson_cache[1] = {(k,mu): poisson.pmf(k=k, mu=2*mu) for k in range(max_count) for mu in kmer_depth_dict.values()}

def cached_poisson_pmf(g,k,mu):
    return min(max(poisson_cache[g-1][(k,mu)], eps), 1-3*eps)

def Probabilities(fam, phases_ch):
    sampleid_mom = family_info.index[fam].mother_sample
    sampleid_dad = family_info.index[fam].father_sample
    sampleid_children = family_info.index[fam].sib_samples

    possible_gs = [(0,0),(0,1), (1,0), (1,1)]
    family_probability_cache = dict()
    
    for k_m in range(max_count): # Iterate through possible k-mer counts for mom.  (up to 30)
        print(k_m/max_count)
        for k_p in range(max_count): # Iterate through possible k-mer counts for dad (up to 30.)
            for k_cs in itertools.combinations_with_replacement(range(max_count), len(phases_ch)): # Iterate through possible k-mer counts for children (up to 30.)
                family_probability_cache[(fam, k_m, k_p, k_cs, phases_ch)] = np.log2(sum(
                    [cached_poisson_pmf(sum(g_m),k_m, kmer_depth_dict[sampleid_mom])*
                     cached_poisson_pmf(sum(g_p), k_p, kmer_depth_dict[sampleid_dad])*
                     np.prod([cached_poisson_pmf(g_m[phase_ch[0]]+g_p[phase_ch[1]], k_c, kmer_depth_dict[id_c]) for id_c, k_c, phase_ch in zip(sampleid_children, k_cs, phases_ch)]
                            ) for g_p in possible_gs for g_m in possible_gs]))
    return family_probability_cache

family_probability_dict = Probabilities(list(all_family_phasings)[N_PHASE])

#with open('/home/groups/dpwall/briannac/alt_haplotypes/intermediate_files/localize/family_probability_cache_%i.pickle' % N_PHASE, 'wb') as f:
#        pickle.dump(family_probability_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

NameError: name 'max_count' is not defined

In [17]:
poisson_cache = [[], []]
avg_kmer_depth=avg_kmer_depth
poisson_cache[0] = {(k,mu): poisson.pmf(k=k, mu=mu) for k in range(max_count) for mu in kmer_depth_dict.values()}
poisson_cache[1] = {(k,mu): poisson.pmf(k=k, mu=2*mu) for k in range(max_count) for mu in kmer_depth_dict.values()}

def cached_poisson_pmf(g,k,mu):
    return min(max(poisson_cache[g-1][(k,mu)], eps), 1-3*eps)

def Probabilities(phases_ch):
    possible_gs = [(0,0),(0,1), (1,0), (1,1)]
    family_probability_cache = dict()
    for k_m in range(max_count): # Iterate through possible k-mer counts for mom.  (up to 30)
        print(k_m/max_count)
        for k_p in range(max_count): # Iterate through possible k-mer counts for dad (up to 30.)
            for k_cs in itertools.combinations_with_replacement(range(max_count), len(phases_ch)): # Iterate through possible k-mer counts for children (up to 30.)
                family_probability_cache[(k_m, k_p, k_cs, phases_ch)] = np.log2(sum(
                    [cached_poisson_pmf(sum(g_m),k_m, kmer_depth_dict)*
                     cached_poisson_pmf(sum(g_p), k_p, )*
                     np.prod([cached_poisson_pmf(g_m[phase_ch[0]]+g_p[phase_ch[1]], k_c) for k_c, phase_ch in zip(k_cs, phases_ch)]
                            ) for g_p in possible_gs for g_m in possible_gs]))
    return family_probability_cache

#pool = mp.Pool(mp.cpu_count())

#dicts = pool.map(Probabilities, list(all_family_phasings))
#pool.close()
family_probability_dict = Probabilities(list(all_family_phasings)[N_PHASE])
#for d in dicts:
#    family_probability_dict.update(d)

with open('/home/groups/dpwall/briannac/alt_haplotypes/intermediate_files/localize/family_probability_cache_%i.pickle' % N_PHASE, 'wb') as f:
        pickle.dump(family_probability_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

In [1]:
avg_kmer_depth = 5.5
eps = np.log2(.0001)
max_count = 5

with open('/home/groups/dpwall/briannac/alt_haplotypes/data/phasings/phased_fams/phased_fams_all.pickle', 'rb') as f:
    phased_fam_dict = pickle.load(f)

all_family_phasings = set()
for f in phased_fam_dict:
    fam_phases = phased_fam_dict[f]
    all_family_phasings = all_family_phasings.union(set([tuple(v) for v in fam_phases.values]))
    
    
poisson_cache = [[], []]
avg_kmer_depth=avg_kmer_depth
poisson_cache[0] = [poisson.pmf(k=k, mu=avg_kmer_depth) for k in range(max_count)]
poisson_cache[1] = [poisson.pmf(k=k, mu=2*avg_kmer_depth) for k in range(max_count)]

def cached_poisson_pmf(k,g):
    return min(max(poisson_cache[g-1][k], eps), 1-3*eps)

def Probabilities(phases_ch):
    possible_gs = [(0,0),(0,1), (1,0), (1,1)]
    family_probability_cache = dict()
    for k_m in range(max_count): # Iterate through possible k-mer counts for mom.  (up to 30)
        for k_p in range(max_count): # Iterate through possible k-mer counts for dad (up to 30.)
            for k_cs in itertools.combinations_with_replacement(range(max_count), len(phases_ch)): # Iterate through possible k-mer counts for children (up to 30.)
                family_probability_cache[(k_m, k_p, k_cs, phases_ch)] = np.log2(sum(
                    [cached_poisson_pmf(k_m,sum(g_m))*
                     cached_poisson_pmf(k_p,sum(g_p))*
                     np.prod([cached_poisson_pmf(k_c, g_m[phase_ch[0]]+g_p[phase_ch[1]]) for k_c, phase_ch in zip(k_cs, phases_ch)]
                            ) for g_p in possible_gs for g_m in possible_gs]))
    return family_probability_cache

pool = mp.Pool(mp.cpu_count())
t = time.time()
results = pool.map(Probabilities, list(all_family_phasings)[:30])
print(time.time()-t)
pool.close()
family_probability_dict = {}
for d in results:
    family_probability_dict.update(d)

with open('/home/groups/dpwall/briannac/alt_haplotypes/intermediate_files/localize/family_probability_cache.pickle', 'wb') as f:
        pickle.dump(family_probability_cache, f, protocol=pickle.HIGHEST_PROTOCOL)

NameError: name 'np' is not defined

In [43]:
#for i,phases_ch in enumerate(all_family_phasings): # Iterate through all children's phasings.
#    print(i/len(all_family_phasings))
def Probabilities(phases_ch):
    possible_gs = [(0,0),(0,1), (1,0), (1,1)]
    family_probability_cache = dict()
    for k_m in range(max_count): # Iterate through possible k-mer counts for mom.  (up to 30)
        for k_p in range(max_count): # Iterate through possible k-mer counts for dad (up to 30.)
            for k_cs in itertools.combinations_with_replacement(range(max_count), len(phases_ch)): # Iterate through possible k-mer counts for children (up to 30.)
                family_probability_cache[(k_m, k_p, k_cs, phases_ch)] = np.log2(sum(
                    [cached_poisson_pmf(k_m,sum(g_m))*
                     cached_poisson_pmf(k_p,sum(g_p))*
                     np.prod([cached_poisson_pmf(k_c, g_m[phase_ch[0]]+g_p[phase_ch[1]]) for k_c, phase_ch in zip(k_cs, phases_ch)]
                            ) for g_p in possible_gs for g_m in possible_gs]))
    return family_probability_cache

pool = mp.Pool(mp.cpu_count())
t = time.time()
results = pool.map(Probabilities, list(all_family_phasings)[:30])
print(time.time()-t)
pool.close()
family_probability_dict = {}
for d in results:
    family_probability_dict.update(d)


In [32]:
pool = mp.Pool(mp.cpu_count())

In [44]:
t = time.time()
results = [Probabilities(phases_ch) for phases_ch in list(all_family_phasings)[:10]]
print(time.time()-t)

4.612492084503174


In [48]:
pool = mp.Pool(mp.cpu_count())
t = time.time()
results = pool.map(Probabilities, list(all_family_phasings)[:30])
print(time.time()-t)
pool.close()
family_probability_dict = {}
for d in results:
    family_probability_dict.update(d)


4.258832216262817


In [90]:
all_family_phasings = set()
for f in phased_fam_dict:
    try:
        fam_phases = phased_fam_dict[f]
        phasings = set([tuple(v) for v in fam_phases.values])
        k_counts = tuple([round(k/3)*3 for k in [kmer_depth_dict[family_info.loc[f].mother_sample],
                              kmer_depth_dict[family_info.loc[f].father_sample]] + 
                              [kmer_depth_dict[s] for s in family_info.loc[f].sib_samples]])
        all_family_phasings = all_family_phasings.union({(k_counts, p) for p in phasings})
    except: print("Not all IDs present in ", f)
print(len(all_family_phasings))

Not all IDs present in  AU1860
Not all IDs present in  AU1308
Not all IDs present in  AU3051
Not all IDs present in  AU3154
Not all IDs present in  AU2410
Not all IDs present in  AU3053
1036


In [97]:
poisson_cache = [[], []]
poisson_cache[0] = {(k,mu):poisson.pmf(k=k, mu=mu) for mu in set([round(v/1.0)*1.0 for v in kmer_depth_dict.values()]) for k in range(int(np.floor(1.0*mu)))}
poisson_cache[1] = {(k,mu):poisson.pmf(k=k, mu=2*mu) for mu in set([round(v/1.0)*1.0 for v in kmer_depth_dict.values()]) for k in range(int(np.floor(1.0*mu)))}

In [98]:
len(poisson_cache[0])

85

In [96]:
def cached_poisson_pmf(g,k,mu):
    return min(max(poisson_cache[g-1][(k,mu)], eps), 1-3*eps)

phases_ch  = list(all_family_phasings)[N_PHASE]
phases = phases_ch[1]
k_depth_mom = phases_ch[0][0]
k_depth_dad = phases_ch[0][1]
k_depth_ch = phases_ch[0][2:]

possible_gs = [(0,0),(0,1), (1,0), (1,1)]
family_probability_cache = dict()
for k_m in range(int(np.ceil(3*phases_ch[0][0]))): # Iterate through possible k-mer counts for mom.  (up to 30)
    print(k_m/int(np.ceil(3*phases_ch[0][0])))
    for k_p in range(int(np.ceil(3*phases_ch[0][1]))): # Iterate through possible k-mer counts for dad (up to 30.)
        print('kp:', k_p/int(np.ceil(3*phases_ch[0][1])), len(family_probability_cache))
        for k_cs in itertools.product(*[[i for i in range(int(np.ceil(3*k_depth)))] for k_depth in k_depth_ch]): # Iterate through possible k-mer counts for children (up to 30.)
            family_probability_cache[(k_m, k_p, k_cs, phases_ch)] = np.log2(sum(
                    [cached_poisson_pmf(sum(g_m),k_m, k_depth_mom)*
                     cached_poisson_pmf(sum(g_p), k_p, k_depth_dad )*
                     np.prod([cached_poisson_pmf(g_m[phase_ch[0]]+g_p[phase_ch[1]], k_c, mu) for k_c, phase_ch, mu in zip(k_cs, phases, k_depth_ch)]
                            ) for g_p in possible_gs for g_m in possible_gs]))
family_probability_dict =  family_probability_cache

#pool = mp.Pool(mp.cpu_count())
#dicts = pool.map(Probabilities, list(all_family_phasings))
#pool.close()
#family_probability_dict = Probabilities(list(all_family_phasings)[N_PHASE])
#for d in dicts:
#    family_probability_dict.update(d)

with open('/home/groups/dpwall/briannac/alt_haplotypes/intermediate_files/localize/family_probability_cache_%i.pickle' % N_PHASE, 'wb') as f:
        pickle.dump(family_probability_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

0.0
kp: 0.0 0


KeyError: (6, 6.0)

In [76]:
len([i for i in itertools.product(*[[i for i in range(int(np.ceil(3*k_depth)))] for k_depth in k_depth_ch])])

104976